# The Statistical Whitening Transform 
In a number of modeling scenarios, it is beneficial to transform the to-be-modeled data such that it has an identity covariance matrix, a procedure known as Statistical Whitening. When data have an identity covariance, all dimensions are statistically independent, and the variance of the data along each of the dimensions is equal to one.
Enforcing statistical independence is useful for a number of reasons. For example, in probabilistic models of data that exist in multiple dimensions, the joint distribution–which may be very complex and difficult to characterize–can factorize into a product of many simpler distributions when the dimensions are statistically independent. Forcing all dimensions to have unit variance is also useful. For instance, scaling all variables to have the same variance treats each dimension with equal importance.

## Decorrelation: Transforming Data to Have a Diagonal Covariance Matrix
Let’s say we have some data matrix X composed of K dimensions and n observations (X has  size $[K \times n]$).  Let’s also assume that the rows of X have been centered (the mean has been subracted across all observations) . The covariance $\Sigma$ of each of the dimensions with respect to the other is
$$\Sigma = Cov(X) = \mathbb E[X X^T] $$   
Where the covariance $\mathbb E[X X^T] $ can be estimated from the data matrix as follows:
$$\mathbb E[X X^T] \approx \frac{X X^T}{n}   $$  
The covariance matrix $\Sigma$, by definition is symmetric and positive semi-definite. Thus we can write the matrix as the product of two simpler matrices E and D, using a procedure known as Eigenvalue Decomposition:
$$\Sigma = EDE^{-1}$$                            
The matrix E is an $[K \times K]$-sized matrix, where each column is an eigenvector of $\Sigma$, and D is a diagonal matrix whose diagonal elements $D_{ii}$ are eigenvalues that correspond to the eigenvectors of the i-th column of E. We can transform $\Sigma$ into the diagonal matrix D
$$E^{-1} \Sigma E = D $$          
Now, imagine the goal is to transform the data matrix X into a new data matrix Y
$$Y = W_DX$$                   
whose dimensions are uncorrelated (i.e. Y has a diagonal covariance D). Thus we want to determine the transformation $W_D$ that makes:
$$D = Cov(Y) = \mathbb E[YY^T]    $$          
Using the earlier expressions, we get
$$ D = \frac{W_DX(W_DX)^T}{n}          $$ 
$$D = W_D W_D^T \Sigma     $$             
$$ E^{-1}\Sigma E = W_D W_D^T \Sigma     $$          
    $$    \Sigma^{-1}E^{-1} \Sigma E = \Sigma^{-1}W_D W_D^T \Sigma$$
now, because $E^{-1} = E^T$          
        $$    E^TE = W_DW_D^T $$
 $$  W_D = E^T                                 $$    
This means that we can transform X into an uncorrelated (i.e. orthogonal) set of variables by premultiplying data matrix X with the transpose of the the eigenvectors of data covariance matrix $\Sigma.$

## Whitening: Transforming data to have an Identity Covariance matrix
Ok, so now we have a way of transforming our data so that the dimensions are uncorrelated. However, this only gives us a diagonal covariance matrix, not an Identity covariance matrix. In order to obtain an Identity covariance, we also need to scale each dimension so that its variance is equal to one. How can we determine this transformation? We know how to transform our data so that the covariance is equal to D. If we can determine the transformation that leaves D = I, then we can apply this transformation to our decorrelated covariance to give us the desired whitening transform. We can determine this from the somewhat trivial notion that
$$D^{-1}D = I   $$                      
and further that
$$D^{-1} = D^{-1/2}ID^{-1/2}   $$    
Now, from earlier expressions, we can see that
$$D^{-1/2}E^{-1}\Sigma E D^{-1/2} = I       $$  
Now say that we define a variable $Y = W_W X $, where $W_W$ is the desired whitening transform, that leaves the covariance of Y equal to the identity matrix. Using essentially the same set of derivation steps as above to solve for $W_D$, we find that
$$W_W = D^{-1/2}E^T   $$ 
$$= D^{-1/2}W_D      $$  
Thus, the whitening transform is simply the decorrelation transform, but scaled by the inverse of the square root of the D (here the inverse and square root can be performed element-wise because D is a diagonal matrix).

In [1]:
import pandas as pd
import numpy as np
import pylab


## Interpretation of the Whitening Transform
So what does the whitening transformation actually do to the data? We investigate this transformation below: The first operation decorrelates the data by premultiplying the data with the eigenvector matrix $E^T$, calculated from the data covariance. This decorrelation can be thought of as a rotation that reorients the data so that the principal axes of the data are aligned with the axes along which the data has the largest (orthogonal) variance. This rotation is essentially the same procedure as the oft-used Principal Components Analysis (PCA).

The second operation, scaling by $D^{-1/2}$ can be thought of squeezing the data–if the variance along a dimension is larger than one–or stretching the data–if the variance along a dimension is less than one. The stretching and squeezing forms the data into a sphere about the origin (which is why whitening is also referred to as “sphering”). 

from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train.shape